In [1]:
minimum_precision = 4
# THIS IS AN INCLUSIVE RANGE!!
maximum_precision = 16
bits_per_registry = 6
registry_per_word = (32 // bits_per_registry)

In [2]:
test = """
use hyperloglog_rs::prelude::*;

#[test]
pub fn test_hyper_log_log_at_precision_{precision}() {{
    const PRECISION: usize = {precision};
    
    for elements in [5, 10, 15, 100, 200, 1000, 10_000, 100_000, 1_000_000]{{
        let mut hll: HyperLogLog<PRECISION> = HyperLogLog::new();

        for i in 0..elements {{
            hll.insert(i);
        }}

        assert!(
            hll.count_dispatched() >= elements as f32 * 7.0_f32 / 10.0_f32,
            "Obtained: {{}}, Expected around: {{}}",
            hll.count_dispatched(), elements
        );

        assert!(
            hll.count() >= elements as f32 * 7.0_f32 / 10.0_f32,
            "Obtained: {{}}, Expected around: {{}}",
            hll.count(), elements
        );

        assert!(
            hll.count_dispatched() <= elements as f32 * 14.0_f32 / 10.0_f32,
            "Obtained: {{}}, Expected around: {{}}",
            hll.count_dispatched(), elements
        );

        assert!(
            hll.count() <= elements as f32 * 14.0_f32 / 10.0_f32,
            "Obtained: {{}}, Expected around: {{}}",
            hll.count(), elements
        );
    }}
}}
"""

In [3]:
for precision in range(minimum_precision, 13):
    with open(f"tests/test_hll_{precision}.rs", "w") as f:
        f.write(test.format(
            precision=precision
        ))